# Redo Marks fitness pred

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import sys
from sklearn.metrics import roc_auc_score

sys.path.append("../")
from utils import *
from global_variables import *
from escape_map import *
pgm_path = "PGM/"
sys.path.append(pgm_path + "source/")
sys.path.append(pgm_path + "utilities/")
import utilities, Proteins_utils, sequence_logo, plots_utils
import rbm, RBM_utils

E:\ESCAPE_MAP_DRAFT\PGM\source\numba_utilities.py:1124: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float32, 2, 'F', False, aligned=True), Array(float32, 2, 'A', False, aligned=True))
  dmean_v_dw = np.dot(s1.T, V)
E:\ESCAPE_MAP_DRAFT\PGM\source\numba_utilities.py:961: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float32, 1, 'A', False, aligned=True), Array(float32, 2, 'A', False, aligned=True))
  mean_V = np.dot(weights, V) / sum_weights


Loaded 29 KD vectors


C:\Users\maria\AppData\Roaming\Python\Python312\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:

# --- Load ---
df = pd.read_csv("rbd_dist_one_scores_gisaid.csv")
df=df[df['i']>=349] 
df=df[df['i']<=526]



In [3]:
df

,Unnamed: 0,i,wt,mut,bloom_ace2_binding,bloom_expression,chan_expression,chan_ace2_binding,fitness_eve,dissimilarity_charge_hydro,accessibility_wcn,evescape,max_escape_experiment_bloom,is_escape_experiment_bloom,max_escape_experiment_xie,is_escape_experiment_xie,is_escape_experiment_all,mutation,counts,first_seen
135,135,349,S,A,-0.34,-1.13,-3.83,-2.68,-6.902246,-1.222299,-1.417140,-2.387840,0.00000,False,0.000000,False,False,S349A,53.0,NaN
136,136,349,S,C,-0.12,-2.03,-1.77,-0.78,-7.368066,-1.609191,-1.417140,-2.563646,0.00000,False,0.000000,False,False,S349C,10.0,NaN
137,137,349,S,F,-2.03,-2.50,-3.46,-2.75,-7.207227,-0.559057,-1.417140,-2.345080,0.00000,False,0.000000,False,False,S349F,16.0,NaN
138,138,349,S,G,-0.59,-2.08,-1.04,-0.77,-6.908368,-1.388110,-1.417140,-2.418569,0.00000,False,0.000000,False,False,S349G,NaN,NaN
139,139,349,S,I,-4.25,-2.69,-3.35,-1.95,-6.500989,-0.337976,-1.417140,-2.156518,0.00000,False,0.000000,False,False,S349I,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,1531,526,G,E,-0.01,-0.13,NaN,NaN,-8.738202,1.067187,-1.885148,-3.535322,0.07851,False,0.010790,False,False,G526E,NaN,NaN
1532,1532,526,G,R,-0.08,-0.30,NaN,NaN,-8.578857,3.241148,-1.885148,-3.269983,0.04760,False,0.008869,False,False,G526R,6.0,NaN
1533,1533,526,G,S,-0.02,-0.14,NaN,NaN,-8.599188,-1.388110,-1.885148,-3.868234,0.05173,False,0.008869,False,False,G526S,NaN,NaN
1534,1534,526,G,V,-0.04,-0.12,NaN,NaN,-8.664563,-1.461803,-1.885148,-3.899844,0.05142,False,0.008869,False,False,G526V,5.0,NaN


In [4]:
Y_ALL = (df["first_seen"].notna()).astype(int).to_numpy()
scores_baseline=df['evescape'].to_numpy()
#average of y
print(Y_ALL.sum()/len(Y_ALL))

0.25767309064953603


In [5]:
WT

'SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCG'

In [6]:
def get_sequence(WT, site,	mutation):
    #site is expected to be 349 instead of 0
    site=site-349
    return WT[:site]+mutation+WT[site+1:]

#example:

df['sequence']=df.apply(lambda x: get_sequence(WT, x['i'],  x['mut']), axis=1)
    

In [7]:
with open('single_variants.fasta', 'w') as f:
    for index, row in df.iterrows():
        f.write('>' + str(index) + '\n')
        f.write(row['sequence'] + '\n')
seqs=Proteins_utils.load_FASTA('single_variants.fasta', drop_duplicates=False)

# EscapeMap scoring

In [9]:
raw_conc = np.full(len(KD_VECTORS), -8.0, dtype=np.float64)  # antibodies
model = EscapeMap(
    rbm=RBM,
    kd_vectors=KD_VECTORS,
    ace2_vector=ACE2_KD_VECTOR,
    raw_concentrations=raw_conc,
    raw_ace2=-9,      # ACE2 concentration in log10 space
    raw_beta=-1,
)
scores=-model(seqs)
#auc to compare score and y
auc = roc_auc_score(Y_ALL, scores)
print("AUC:", auc)
#params are the individual concentrations for each Ab (dont put -12 everywhere), raw_ace2, raw_beta
# i want to find best parameters to maximize predictive power=auc, but could be something else


AUC: 0.6440163541444706


In [10]:
wanted = ["S309", "LY-CoV555", "REGN10987", "REGN10933"]

# strict: fail if any key is missing
for k in wanted:
    assert k in KD_VECTORS, f"Missing key: {k}"
kd_vector_small = {k: KD_VECTORS[k] for k in wanted}


In [12]:
auc = roc_auc_score(Y_ALL, scores_baseline)
print("AUC Evescape:", auc)

AUC Evescape: 0.6318000213083315


In [14]:
from escape_map_torch import EscapeMapTorch
raw_conc = np.full(len(KD_VECTORS), -8.0, dtype=np.float64)  # antibodies
model = EscapeMapTorch(
    rbm=RBM,
    kd_vectors=KD_VECTORS,
    ace2_vector=ACE2_KD_VECTOR,
    raw_concentrations=raw_conc,
    raw_ace2=-9,      # ACE2 concentration in log10 space
    raw_beta=-1,
)
scores=-model(seqs)
print('scores shape:', scores)
#auc to compare score and y

auc = roc_auc_score(Y_ALL, scores.detach().numpy())
print("AUC:", auc)
#params are the individual concentrations for each Ab (dont put -12 everywhere), raw_ace2, raw_beta
# i want to find best parameters to maximize predictive power=auc, but could be something else


scores shape: tensor([578.0441, 574.5410, 574.5409,  ..., 576.8531, 576.6968, 572.7727],
       dtype=torch.float64, grad_fn=<NegBackward0>)
AUC: 0.6440163541444706


In [17]:
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from math import ceil
# assumes EscapeMapTorch is defined as above, and KD_VECTORS, ACE2_KD_VECTOR, RBM, WT_SEQ exist

# ----- loss: pairwise logistic (AUC surrogate) -----
def pairwise_logistic_loss(scores: torch.Tensor, y: torch.Tensor, margin: float = 0.0, max_pairs: int = 200_000):
    # scores: [B], y: [B] in {0,1}
    pos = scores[y == 1]
    neg = scores[y == 0]
    if pos.numel() == 0 or neg.numel() == 0:
        return None  # skip batch with single class
    # all pairwise diffs
    # sample pairs if too many
    num_pairs = pos.numel() * neg.numel()
    if num_pairs > max_pairs:
        k = max_pairs
        ip = torch.randint(0, pos.numel(), (k,), device=scores.device)
        ineg = torch.randint(0, neg.numel(), (k,), device=scores.device)
        diff = pos[ip] - neg[ineg]
    else:
        diff = pos[:, None] - neg[None, :]
    # loss = log(1 + exp(-(diff - margin))) = softplus(-(diff - margin))
    return F.softplus(-(diff - margin)).mean()

# ----- optional: BCE loss (logistic classification) -----
bce = torch.nn.BCEWithLogitsLoss()

# ===== data =====
# seqs: np.ndarray [N, L] int; y: np.ndarray [N] in {0,1}
# provide your arrays:
# seqs = ...
# y = ...

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float64  # to match original

model = EscapeMapTorch(
    rbm=RBM, kd_vectors=KD_VECTORS, ace2_vector=ACE2_KD_VECTOR,
    raw_concentrations=None, raw_ace2=None, raw_beta=None,
    total_beta=1.0, device=device, dtype=dtype
).to(device)

# optimize only the three parameters
opt = torch.optim.Adam(
    [model.raw_concentrations, model.raw_beta, model.raw_ace2],
    lr=2e-1, weight_decay=1e-5
)

# ===== training =====
X = torch.as_tensor(seqs, dtype=torch.long, device=device)
Y = torch.as_tensor(Y_ALL, dtype=dtype, device=device)

epochs = 20
batch_size = 256
use_pairwise = True  # set False to train with BCE

N = X.size(0)
idx_all = torch.arange(N, device=device)

for epoch in range(1, epochs + 1):
    # shuffle
    perm = idx_all[torch.randperm(N, device=device)]
    for bi in range(ceil(N / batch_size)):
        bidx = perm[bi * batch_size : (bi + 1) * batch_size]
        xb = X[bidx]
        yb = Y[bidx]

        opt.zero_grad()
        scores = -model(xb)  # higher score = more positive
        if use_pairwise:
            loss = pairwise_logistic_loss(scores, yb, margin=0.0)
            if loss is None:
                # fallback to BCE if only one class in batch
                loss = bce(scores, yb)
        else:
            loss = bce(scores, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(
            [model.raw_concentrations, model.raw_beta, model.raw_ace2], max_norm=10.0
        )
        opt.step()

    # evaluate AUC on full set
    with torch.no_grad():
        full_scores = -model(X).detach().cpu().numpy()
    auc = roc_auc_score(Y_ALL, full_scores)
    print(f"epoch {epoch:03d} | loss {float(loss):.4f} | AUC {auc:.4f}")

# ===== final scores & AUC =====
with torch.no_grad():
    scores = -model(X).cpu().numpy()
final_auc = roc_auc_score(y, scores)
print("Final AUC:", final_auc)

# access learned params
rc = model.raw_concentrations.detach().cpu().numpy()
rb = float(model.raw_beta.detach().cpu())
ra = float(model.raw_ace2.detach().cpu())
print("raw_concentrations shape:", rc)
print("raw_beta:", rb, "raw_ace2:", ra)


epoch 001 | loss 0.5743 | AUC 0.6825
epoch 002 | loss 0.6054 | AUC 0.6836
epoch 003 | loss 0.5646 | AUC 0.6886
epoch 004 | loss 0.6122 | AUC 0.6967
epoch 005 | loss 0.5720 | AUC 0.7044
epoch 006 | loss 0.5983 | AUC 0.7088


KeyboardInterrupt: 

In [ ]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
import numpy as np
import pandas as pd

# 1) parse the date column
col = "first seen" if "first seen" in df.columns else "first_seen"
df["first_seen_dt"] = pd.to_datetime(df[col], format="%Y-%m", errors="coerce")

# 2) 6-month periods starting 2021-01-01
start = pd.Timestamp("2020-01-01")
end = df["first_seen_dt"].max(skipna=True)
print("Date range:", start, "to", end)
check_starts = pd.date_range(start, end, freq="12MS")              # 2021-01, 2021-07, ...
check_ends = check_starts + pd.DateOffset(months=12)

# 3) 1 if first_seen is within the 6-month window [start, end)
dates = df["first_seen_dt"].values[:, None]                       # (N,1) datetime64[ns]
starts = check_starts.values[None, :]                             # (1,M)
ends = check_ends.values[None, :]                                 # (1,M)
valid = (~pd.isna(df["first_seen_dt"]).values)[:, None]           # (N,1) bool
y_mat = (valid & (dates >= starts) & (dates < ends)).astype(int)  # (N,M)

# 4) attach as columns: y_YYYY-MM (period start)
y_df = pd.DataFrame(
    y_mat,
    index=df.index,
    columns=[f"y_{d.strftime('%Y-%m')}" for d in check_starts]
)
print("Last date of first_seen:", df["first_seen_dt"].max())
df_out = pd.concat([df, y_df], axis=1)


Date range: 2020-01-01 00:00:00 to 2022-08-01 00:00:00
Last date of first_seen: 2022-08-01 00:00:00


In [19]:
# import numpy as np
# import pandas as pd
# import torch
# import torch.nn.functional as F
# from sklearn.metrics import roc_auc_score
# from math import ceil

# # pairwise AUC surrogate
# def pairwise_logistic_loss(scores: torch.Tensor, y: torch.Tensor, margin: float = 0.0, max_pairs: int = 200_000):
#     pos = scores[y == 1]
#     neg = scores[y == 0]
#     if pos.numel() == 0 or neg.numel() == 0:
#         return None
#     num_pairs = pos.numel() * neg.numel()
#     if num_pairs > max_pairs:
#         k = max_pairs
#         ip = torch.randint(0, pos.numel(), (k,), device=scores.device)
#         ineg = torch.randint(0, neg.numel(), (k,), device=scores.device)
#         diff = pos[ip] - neg[ineg]
#     else:
#         diff = pos[:, None] - neg[None, :]
#     return F.softplus(-(diff - margin)).mean()

# bce = torch.nn.BCEWithLogitsLoss()

# def optimize_for_check_dates(
#     df, seqs,
#     check_prefix="y_", lr=2e-1, weight_decay=1e-5,
#     epochs=30, batch_size=1024, use_pairwise=True,
#     device=None, dtype=torch.float64,
#     params_csv="fitted_params.csv", scores_csv="scores_by_date.csv"
# ):
#     device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     check_cols = sorted([c for c in df.columns if c.startswith(check_prefix)])
#     ab_names = list(kd_vector_small.keys())

#     X = torch.as_tensor(seqs, dtype=torch.long, device=device)

#     param_rows = []
#     score_rows = []

#     for col in check_cols:
#         y = df[col].to_numpy().astype(int)
#         if y.sum() == 0 or y.sum() == y.size:
#             print(f"{col}: skipped (single class)")
#             continue

#         Y = torch.as_tensor(y, dtype=dtype, device=device)

#         model = EscapeMapTorch(
#             rbm=RBM, kd_vectors=kd_vector_small, ace2_vector=ACE2_KD_VECTOR,
#             raw_concentrations=None, raw_ace2=None, raw_beta=None,
#             total_beta=1.0, device=device, dtype=dtype
#         ).to(device)

#         opt = torch.optim.Adam(
#             [model.raw_concentrations, model.raw_beta, model.raw_ace2],
#             lr=lr, weight_decay=weight_decay
#         )

#         N = X.size(0)
#         idx_all = torch.arange(N, device=device)

#         for _ in range(epochs):
#             perm = idx_all[torch.randperm(N, device=device)]
#             for bi in range(ceil(N / batch_size)):
#                 bidx = perm[bi * batch_size : (bi + 1) * batch_size]
#                 xb = X[bidx]
#                 yb = Y[bidx]

#                 opt.zero_grad()
#                 scores_b = -model(xb)
#                 if use_pairwise:
#                     loss = pairwise_logistic_loss(scores_b, yb, margin=0.0)
#                     if loss is None:
#                         loss = bce(scores_b, yb)
#                 else:
#                     loss = bce(scores_b, yb)
#                 loss.backward()
#                 torch.nn.utils.clip_grad_norm_(
#                     [model.raw_concentrations, model.raw_beta, model.raw_ace2], max_norm=10.0
#                 )
#                 opt.step()

#         with torch.no_grad():
#             scores_full = -model(X).cpu().numpy()
#             auc = roc_auc_score(y, scores_full)
#             rc = model.raw_concentrations.detach().cpu().numpy()
#             rb = float(model.raw_beta.detach().cpu())
#             ra = float(model.raw_ace2.detach().cpu())

#         gm_c = float(10.0 ** (np.mean(rc)))
#         gm_beta = float(np.exp(rb))
#         check_date = col[len(check_prefix):]

#         # parameters row
#         row = {
#             "check_date": check_date,
#             "raw_beta": rb,
#             "raw_ace2": ra,
#             "AUC": auc,
#         }
#         # per-antibody raw_c columns
#         for name, val in zip(ab_names, rc):
#             row[f"raw_c_{name}"] = float(val)
#         param_rows.append(row)

#         # scores rows
#         score_rows.extend(
#             {"check_date": check_date, "seq_index": int(i), "score": float(s), "y": int(y[i])}
#             for i, s in enumerate(scores_full)
#         )

#         print(f"{col}: AUC={auc:.4f} raw_ace2={ra:.6g} gm_c={gm_c:.6g} gm_beta={gm_beta:.6g}")

#     params_df = pd.DataFrame(param_rows)
#     scores_df = pd.DataFrame(score_rows)

#     params_df.to_csv(params_csv, index=False)
#     scores_df.to_csv(scores_csv, index=False)

#     return params_df, scores_df

# # Example:
# params_df, scores_df = optimize_for_check_dates(df_out, seqs, params_csv="params_4ab.csv", scores_csv="scores_4ab.csv")


In [20]:
# import numpy as np
# import pandas as pd
# import torch
# import torch.nn.functional as F
# from sklearn.metrics import roc_auc_score
# from math import ceil

# # pairwise AUC surrogate
# def pairwise_logistic_loss(scores: torch.Tensor, y: torch.Tensor, margin: float = 0.0, max_pairs: int = 200_000):
#     pos = scores[y == 1]
#     neg = scores[y == 0]
#     if pos.numel() == 0 or neg.numel() == 0:
#         return None
#     num_pairs = pos.numel() * neg.numel()
#     if num_pairs > max_pairs:
#         k = max_pairs
#         ip = torch.randint(0, pos.numel(), (k,), device=scores.device)
#         ineg = torch.randint(0, neg.numel(), (k,), device=scores.device)
#         diff = pos[ip] - neg[ineg]
#     else:
#         diff = pos[:, None] - neg[None, :]
#     return F.softplus(-(diff - margin)).mean()

# bce = torch.nn.BCEWithLogitsLoss()

# def optimize_for_check_dates(
#     df, seqs,
#     check_prefix="y_", lr=2e-1, weight_decay=1e-5,
#     epochs=30, batch_size=1024, use_pairwise=True,
#     device=None, dtype=torch.float64,
#     params_csv="fitted_params.csv", scores_csv="scores_by_date.csv"
# ):
#     device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     check_cols = sorted([c for c in df.columns if c.startswith(check_prefix)])
#     ab_names = list(KD_VECTORS.keys())

#     X = torch.as_tensor(seqs, dtype=torch.long, device=device)

#     param_rows = []
#     score_rows = []

#     for col in check_cols:
#         y = df[col].to_numpy().astype(int)
#         if y.sum() == 0 or y.sum() == y.size:
#             print(f"{col}: skipped (single class)")
#             continue

#         Y = torch.as_tensor(y, dtype=dtype, device=device)

#         model = EscapeMapTorch(
#             rbm=RBM, kd_vectors=KD_VECTORS, ace2_vector=ACE2_KD_VECTOR,
#             raw_concentrations=None, raw_ace2=None, raw_beta=None,
#             total_beta=1.0, device=device, dtype=dtype
#         ).to(device)

#         opt = torch.optim.Adam(
#             [model.raw_concentrations, model.raw_beta, model.raw_ace2],
#             lr=lr, weight_decay=weight_decay
#         )

#         N = X.size(0)
#         idx_all = torch.arange(N, device=device)

#         for _ in range(epochs):
#             perm = idx_all[torch.randperm(N, device=device)]
#             for bi in range(ceil(N / batch_size)):
#                 bidx = perm[bi * batch_size : (bi + 1) * batch_size]
#                 xb = X[bidx]
#                 yb = Y[bidx]

#                 opt.zero_grad()
#                 scores_b = -model(xb)
#                 if use_pairwise:
#                     loss = pairwise_logistic_loss(scores_b, yb, margin=0.0)
#                     if loss is None:
#                         loss = bce(scores_b, yb)
#                 else:
#                     loss = bce(scores_b, yb)
#                 loss.backward()
#                 torch.nn.utils.clip_grad_norm_(
#                     [model.raw_concentrations, model.raw_beta, model.raw_ace2], max_norm=10.0
#                 )
#                 opt.step()

#         with torch.no_grad():
#             scores_full = -model(X).cpu().numpy()
#             auc = roc_auc_score(y, scores_full)
#             rc = model.raw_concentrations.detach().cpu().numpy()
#             rb = float(model.raw_beta.detach().cpu())
#             ra = float(model.raw_ace2.detach().cpu())

#         gm_c = float(10.0 ** (np.mean(rc)))
#         gm_beta = float(np.exp(rb))
#         check_date = col[len(check_prefix):]

#         # parameters row
#         row = {
#             "check_date": check_date,
#             "raw_beta": rb,
#             "raw_ace2": ra,
#             "AUC": auc,
#         }
#         # per-antibody raw_c columns
#         for name, val in zip(ab_names, rc):
#             row[f"raw_c_{name}"] = float(val)
#         param_rows.append(row)

#         # scores rows
#         score_rows.extend(
#             {"check_date": check_date, "seq_index": int(i), "score": float(s), "y": int(y[i])}
#             for i, s in enumerate(scores_full)
#         )

#         print(f"{col}: AUC={auc:.4f} raw_ace2={ra:.6g} gm_c={gm_c:.6g} gm_beta={gm_beta:.6g}")

#     params_df = pd.DataFrame(param_rows)
#     scores_df = pd.DataFrame(score_rows)

#     params_df.to_csv(params_csv, index=False)
#     scores_df.to_csv(scores_csv, index=False)

#     return params_df, scores_df

# # Example:
# params_df, scores_df = optimize_for_check_dates(df_out, seqs, params_csv="params.csv", scores_csv="scores.csv")


# one fit only

In [21]:
# import numpy as np
# import pandas as pd
# import torch
# import torch.nn.functional as F
# from sklearn.metrics import roc_auc_score
# from math import ceil

# # pairwise AUC surrogate
# def pairwise_logistic_loss(scores: torch.Tensor, y: torch.Tensor, margin: float = 0.0, max_pairs: int = 200_000):
#     pos = scores[y == 1]
#     neg = scores[y == 0]
#     if pos.numel() == 0 or neg.numel() == 0:
#         return None
#     num_pairs = pos.numel() * neg.numel()
#     if num_pairs > max_pairs:
#         k = max_pairs
#         ip = torch.randint(0, pos.numel(), (k,), device=scores.device)
#         ineg = torch.randint(0, neg.numel(), (k,), device=scores.device)
#         diff = pos[ip] - neg[ineg]
#     else:
#         diff = pos[:, None] - neg[None, :]
#     return F.softplus(-(diff - margin)).mean()

# bce = torch.nn.BCEWithLogitsLoss()

# def optimize_for_check_dates(
#     df, seqs,
#     check_prefix="y_", lr=2e-1, weight_decay=1e-5,
#     epochs=30, batch_size=1024, use_pairwise=True,
#     device=None, dtype=torch.float64,
#     params_csv="fitted_params.csv", scores_csv="scores_by_date.csv"
# ):
#     device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     check_cols = sorted([c for c in df.columns if c.startswith(check_prefix)])
#     ab_names = list(kd_vector_small.keys())

#     X = torch.as_tensor(seqs, dtype=torch.long, device=device)

#     param_rows = []
#     score_rows = []

#     for col in check_cols:
#         y = Y_ALL
#         if y.sum() == 0 or y.sum() == y.size:
#             print(f"{col}: skipped (single class)")
#             continue

#         Y = torch.as_tensor(y, dtype=dtype, device=device)

#         model = EscapeMapTorch(
#             rbm=RBM, kd_vectors=kd_vector_small, ace2_vector=ACE2_KD_VECTOR,
#             raw_concentrations=None, raw_ace2=None, raw_beta=None,
#             total_beta=1.0, device=device, dtype=dtype
#         ).to(device)

#         opt = torch.optim.Adam(
#             [model.raw_concentrations, model.raw_beta, model.raw_ace2],
#             lr=lr, weight_decay=weight_decay
#         )

#         N = X.size(0)
#         idx_all = torch.arange(N, device=device)

#         for _ in range(epochs):
#             perm = idx_all[torch.randperm(N, device=device)]
#             for bi in range(ceil(N / batch_size)):
#                 bidx = perm[bi * batch_size : (bi + 1) * batch_size]
#                 xb = X[bidx]
#                 yb = Y[bidx]

#                 opt.zero_grad()
#                 scores_b = -model(xb)
#                 if use_pairwise:
#                     loss = pairwise_logistic_loss(scores_b, yb, margin=0.0)
#                     if loss is None:
#                         loss = bce(scores_b, yb)
#                 else:
#                     loss = bce(scores_b, yb)
#                 loss.backward()
#                 torch.nn.utils.clip_grad_norm_(
#                     [model.raw_concentrations, model.raw_beta, model.raw_ace2], max_norm=10.0
#                 )
#                 opt.step()

#         with torch.no_grad():
#             scores_full = -model(X).cpu().numpy()
#             auc = roc_auc_score(y, scores_full)
#             rc = model.raw_concentrations.detach().cpu().numpy()
#             rb = float(model.raw_beta.detach().cpu())
#             ra = float(model.raw_ace2.detach().cpu())

#         gm_c = float(10.0 ** (np.mean(rc)))
#         gm_beta = float(np.exp(rb))
#         check_date = col[len(check_prefix):]

#         # parameters row
#         row = {
#             "check_date": check_date,
#             "raw_beta": rb,
#             "raw_ace2": ra,
#             "AUC": auc,
#         }
#         # per-antibody raw_c columns
#         for name, val in zip(ab_names, rc):
#             row[f"raw_c_{name}"] = float(val)
#         param_rows.append(row)

#         # scores rows
#         score_rows.extend(
#             {"check_date": check_date, "seq_index": int(i), "score": float(s), "y": int(y[i])}
#             for i, s in enumerate(scores_full)
#         )

#         print(f"{col}: AUC={auc:.4f} raw_ace2={ra:.6g} gm_c={gm_c:.6g} gm_beta={gm_beta:.6g}")
#         break

#     params_df = pd.DataFrame(param_rows)
#     scores_df = pd.DataFrame(score_rows)

#     params_df.to_csv(params_csv, index=False)
#     scores_df.to_csv(scores_csv, index=False)

#     return params_df, scores_df

# # Example:
# params_df, scores_df = optimize_for_check_dates(df_out, seqs, params_csv="params_4ab_all.csv", scores_csv="scores_4ab_all.csv")


In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from math import ceil

# pairwise AUC surrogate
def pairwise_logistic_loss(scores: torch.Tensor, y: torch.Tensor, margin: float = 0.0, max_pairs: int = 200_000):
    pos = scores[y == 1]
    neg = scores[y == 0]
    if pos.numel() == 0 or neg.numel() == 0:
        return None
    num_pairs = pos.numel() * neg.numel()
    if num_pairs > max_pairs:
        k = max_pairs
        ip = torch.randint(0, pos.numel(), (k,), device=scores.device)
        ineg = torch.randint(0, neg.numel(), (k,), device=scores.device)
        diff = pos[ip] - neg[ineg]
    else:
        diff = pos[:, None] - neg[None, :]
    return F.softplus(-(diff - margin)).mean()

bce = torch.nn.BCEWithLogitsLoss()

def optimize_for_check_dates(
    df, seqs,
    check_prefix="y_", lr=2e-1, weight_decay=1e-5,
    epochs=30, batch_size=1024, use_pairwise=True,
    device=None, dtype=torch.float64,
    params_csv="fitted_params.csv", scores_csv="scores_by_date.csv"
):
    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
    check_cols = sorted([c for c in df.columns if c.startswith(check_prefix)])
    ab_names = list(kd_vector_small.keys())

    X = torch.as_tensor(seqs, dtype=torch.long, device=device)

    param_rows = []
    score_rows = []

    for col in check_cols:
        y = Y_ALL
        if y.sum() == 0 or y.sum() == y.size:
            print(f"{col}: skipped (single class)")
            continue

        Y = torch.as_tensor(y, dtype=dtype, device=device)

        model = EscapeMapTorch(
            rbm=RBM, kd_vectors=kd_vector_small, ace2_vector=ACE2_KD_VECTOR,
            raw_concentrations=None, raw_ace2=None, raw_beta=None,
            total_beta=1.0, device=device, dtype=dtype
        ).to(device)

 
        
        opt = torch.optim.Adam(
            [model.raw_beta, model.raw_ace2],  # omit model.raw_concentrations
            lr=lr, weight_decay=weight_decay
        )

        N = X.size(0)
        idx_all = torch.arange(N, device=device)

        for _ in range(epochs):
            perm = idx_all[torch.randperm(N, device=device)]
            for bi in range(ceil(N / batch_size)):
                bidx = perm[bi * batch_size : (bi + 1) * batch_size]
                xb = X[bidx]
                yb = Y[bidx]

                opt.zero_grad()
                scores_b = -model(xb)
                if use_pairwise:
                    loss = pairwise_logistic_loss(scores_b, yb, margin=0.0)
                    if loss is None:
                        loss = bce(scores_b, yb)
                else:
                    loss = bce(scores_b, yb)
                loss.backward()
                #gradient with respect to concentrations is put to 0
                                # zero gradient for concentrations
    
                torch.nn.utils.clip_grad_norm_(
                    [model.raw_concentrations, model.raw_beta, model.raw_ace2], max_norm=10.0
                )
                opt.step()
                print('model.raw_concentrations:', model.raw_concentrations)

        with torch.no_grad():
            scores_full = -model(X).cpu().numpy()
            auc = roc_auc_score(y, scores_full)
            rc = model.raw_concentrations.detach().cpu().numpy()
            rb = float(model.raw_beta.detach().cpu())
            ra = float(model.raw_ace2.detach().cpu())

        gm_c = float(10.0 ** (np.mean(rc)))
        gm_beta = float(np.exp(rb))
        check_date = col[len(check_prefix):]

        # parameters row
        row = {
            "raw_beta": rb,
            "raw_ace2": ra,
            "AUC": auc,
        }
        # per-antibody raw_c columns
        for name, val in zip(ab_names, rc):
            row[f"raw_c_{name}"] = float(val)
        param_rows.append(row)

        # scores rows
        score_rows.extend(
            {"check_date": check_date, "seq_index": int(i), "score": float(s), "y": int(y[i])}
            for i, s in enumerate(scores_full)
        )

        print(f"{col}: AUC={auc:.4f} raw_ace2={ra:.6g} gm_c={gm_c:.6g} gm_beta={gm_beta:.6g}")
        break

    params_df = pd.DataFrame(param_rows)
    scores_df = pd.DataFrame(score_rows)

    params_df.to_csv(params_csv, index=False)
    scores_df.to_csv(scores_csv, index=False)

    return params_df, scores_df

# Example:
params_df, scores_df = optimize_for_check_dates(df_out, seqs, params_csv="params_noab_all.csv", scores_csv="scores_noab_all.csv")


model.raw_concentrations: Parameter containing:
tensor([-12., -12., -12., -12.], dtype=torch.float64, requires_grad=True)
model.raw_concentrations: Parameter containing:
tensor([-12., -12., -12., -12.], dtype=torch.float64, requires_grad=True)
model.raw_concentrations: Parameter containing:
tensor([-12., -12., -12., -12.], dtype=torch.float64, requires_grad=True)
model.raw_concentrations: Parameter containing:
tensor([-12., -12., -12., -12.], dtype=torch.float64, requires_grad=True)
model.raw_concentrations: Parameter containing:
tensor([-12., -12., -12., -12.], dtype=torch.float64, requires_grad=True)
model.raw_concentrations: Parameter containing:
tensor([-12., -12., -12., -12.], dtype=torch.float64, requires_grad=True)
model.raw_concentrations: Parameter containing:
tensor([-12., -12., -12., -12.], dtype=torch.float64, requires_grad=True)
model.raw_concentrations: Parameter containing:
tensor([-12., -12., -12., -12.], dtype=torch.float64, requires_grad=True)
model.raw_concentrations

In [28]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from math import ceil

# -------- helpers --------

def baseline_margin_loss(scores: torch.Tensor, y: torch.Tensor, baseline: torch.Tensor, margin: float = 0.0):
    """Enforce scores above WT for positives and below WT for negatives."""
    rel = scores - baseline  # relative to WT
    loss = 0.0
    if (y == 1).any():
        pos = rel[y == 1]
        loss = loss + F.softplus(-(pos - margin)).mean()
    if (y == 0).any():
        neg = rel[y == 0]
        loss = loss + F.softplus(neg + margin).mean()
    return loss

def fit_date_with_wt_baseline(
    seqs_np: np.ndarray,
    y_np: np.ndarray,
    kd_vectors: dict,
    rbm,
    ace2_vector,
    epochs: int = 30,
    batch_size: int = 1024,
    lr: float = 2e-1,
    weight_decay: float = 1e-5,
    margin: float = 0.0,
    device=None,
    dtype=torch.float64,
    freeze_conc: bool = True,
):
    """Train EscapeMapTorch for one date with WT-baseline objective."""
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    X = torch.as_tensor(seqs_np, dtype=torch.long, device=device)
    Y = torch.as_tensor(y_np.astype(float), dtype=dtype, device=device)

    model = EscapeMapTorch(
        rbm=rbm, kd_vectors=kd_vectors, ace2_vector=ace2_vector,
        total_beta=1.0, device=device, dtype=dtype
    ).to(device)

    # choose optimizer params
    params = [model.raw_beta, model.raw_ace2] if freeze_conc else [model.raw_concentrations, model.raw_beta, model.raw_ace2]

    opt = torch.optim.Adam(
        [model.raw_concentrations, model.raw_beta, model.raw_ace2],
        lr=lr, weight_decay=weight_decay
    )
    N = X.size(0); idx_all = torch.arange(N, device=device)
    WT = torch.as_tensor(WT_SEQ, dtype=torch.long, device=device)

    for _ in range(epochs):
        perm = idx_all[torch.randperm(N, device=device)]
        for bi in range(ceil(N / batch_size)):
            bidx = perm[bi * batch_size : (bi + 1) * batch_size]
            xb, yb = X[bidx], Y[bidx]

            opt.zero_grad()
            # recompute WT baseline under current params
            wt_baseline = -model(WT)  # scalar
            scores_b = -model(xb)     # [B]
            loss = baseline_margin_loss(scores_b, yb, wt_baseline, margin=margin)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(params, max_norm=10.0)
            opt.step()

    # final evaluation with clipped scores
    with torch.no_grad():
        wt_baseline = -model(WT)
        scores_full = -model(X)
        rel_full = scores_full - wt_baseline
        scores_clipped = torch.clamp_min(rel_full, 0.0).cpu().numpy()
        auc = roc_auc_score(y_np, scores_clipped)
        rc = model.raw_concentrations.detach().cpu().numpy()
        rb = float(model.raw_beta.detach().cpu())
        ra = float(model.raw_ace2.detach().cpu())

    return {
        "scores_clipped": scores_clipped,
        "raw_concentrations": rc,
        "raw_beta": rb,
        "raw_ace2": ra,
        "AUC": auc,
    }

# -------- main driver over dates, save 2 CSVs --------

def optimize_for_check_dates_with_wt(
    df: pd.DataFrame,
    seqs: np.ndarray,
    kd_vectors: dict,
    rbm,
    ace2_vector,
    check_prefix: str = "y_",
    epochs: int = 30,
    batch_size: int = 1024,
    lr: float = 2e-1,
    weight_decay: float = 1e-5,
    margin: float = 0.0,
    device=None,
    dtype=torch.float64,
    params_csv: str = "fitted_params.csv",
    scores_csv: str = "scores_by_date.csv",
    freeze_conc: bool = True,
):
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    check_cols = sorted([c for c in df.columns if c.startswith(check_prefix)])
    ab_names = list(kd_vectors.keys())

    param_rows = []
    score_rows = []

    for col in check_cols:
        y = df[col].to_numpy().astype(int)
        if y.sum() == 0 or y.sum() == y.size:
            print(f"{col}: skipped (single class)")
            continue

        out = fit_date_with_wt_baseline(
            seqs_np=seqs,
            y_np=y,
            kd_vectors=kd_vectors,
            rbm=rbm,
            ace2_vector=ace2_vector,
            epochs=epochs,
            batch_size=batch_size,
            lr=lr,
            weight_decay=weight_decay,
            margin=margin,
            device=device,
            dtype=dtype,
            freeze_conc=freeze_conc,
        )

        check_date = col[len(check_prefix):]
        # params row
        row = {
            "check_date": check_date,
            "raw_beta": out["raw_beta"],
            "raw_ace2": out["raw_ace2"],
            "AUC": out["AUC"],
        }
        for name, val in zip(ab_names, out["raw_concentrations"]):
            row[f"raw_c_{name}"] = float(val)
        param_rows.append(row)

        # scores rows
        score_rows.extend(
            {"check_date": check_date, "seq_index": int(i), "score": float(s), "y": int(y[i])}
            for i, s in enumerate(out["scores_clipped"])
        )

        print(f"{col}: AUC={out['AUC']:.4f} raw_ace2={out['raw_ace2']:.6g} raw_beta={out['raw_beta']:.6g}")

    params_df = pd.DataFrame(param_rows)
    scores_df = pd.DataFrame(score_rows)
    params_df.to_csv(params_csv, index=False)
    scores_df.to_csv(scores_csv, index=False)
    return params_df, scores_df

# -------- run example --------
params_df, scores_df = optimize_for_check_dates_with_wt(
    df=df_out,                  # DataFrame with y_* columns
    seqs=seqs,                  # ndarray [N,L] int
    kd_vectors=kd_vector_small, # or KD_VECTORS
    rbm=RBM,
    ace2_vector=ACE2_KD_VECTOR,
    params_csv="params_pos_4ab_all.csv",
    scores_csv="scores_pos_4ab_all.csv",
    epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5,
    margin=0.0, freeze_conc=True
)


y_2020-01: AUC=0.5646 raw_ace2=-0.0502356 raw_beta=-3.36069
y_2021-01: AUC=0.5349 raw_ace2=-0.0889108 raw_beta=-3.45023
y_2022-01: AUC=0.5350 raw_ace2=-0.417715 raw_beta=-3.55572


In [43]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from math import ceil

# ===== EscapeMapTorch with learnable total_beta =====
class EscapeMapTorch(nn.Module):
    def __init__(
        self, rbm, kd_vectors, ace2_vector,
        raw_concentrations=None, raw_ace2=None, raw_beta=None,
        total_beta=1.0, device=None, dtype=torch.float64
    ):
        super().__init__()
        import math, numpy as np
        self.device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        self.dtype = dtype
        self.kd_vectors = kd_vectors
        self.ace2_vector = {"ace2": ace2_vector}
        self.rbm = rbm
        self.ln10 = math.log(10.0)

        A = len(kd_vectors)
        rc = torch.full((A,), -10.0, dtype=dtype, device=self.device) if raw_concentrations is None \
             else torch.as_tensor(raw_concentrations, dtype=dtype, device=self.device)
        self.raw_concentrations = nn.Parameter(rc)

        ra = -10.0 if raw_ace2 is None else float(raw_ace2)
        rb =  0.0 if raw_beta is None else float(raw_beta)
        self.raw_ace2 = nn.Parameter(torch.tensor(ra, dtype=dtype, device=self.device))
        self.raw_beta = nn.Parameter(torch.tensor(rb, dtype=dtype, device=self.device))

        # NEW: learnable total_beta (positive)
        self.raw_total_beta = nn.Parameter(torch.tensor(float(np.log(total_beta)), dtype=dtype, device=self.device))

    @staticmethod
    def _to_int16_c_contig(x):
        import numpy as np
        if x.dtype != np.int16:
            x = x.astype(np.int16, copy=False)
        return np.ascontiguousarray(x)

    def _get_Kd_batch_numpy(self, seqs_np, vectors, log10=True):
        import numpy as np
        try:
            out = get_Kd(seqs_np, vectors, log10=log10)
        except Exception:
            out = np.stack([get_Kd(x, vectors, log10=log10) for x in seqs_np], axis=0)
        return np.asarray(out)

    @torch.no_grad()
    def _rbm_free_energy(self, seqs_np):
        fe = self.rbm.free_energy(seqs_np)
        return torch.as_tensor(fe, dtype=self.dtype, device=self.device)

    def forward(self, s):
        import numpy as np
        s_np = np.asarray(s)
        single = (s_np.ndim == 1)
        seqs_np = s_np[None, :] if single else s_np
        seqs_np = self._to_int16_c_contig(seqs_np)

        beta = torch.exp(self.raw_beta)
        total_beta = torch.exp(self.raw_total_beta)   # positive
        ln10 = self.ln10

        kds_np = self._get_Kd_batch_numpy(seqs_np, self.kd_vectors, log10=True) * ln10
        kds = torch.as_tensor(np.squeeze(kds_np), dtype=self.dtype, device=self.device)

        kdace2_np = self._get_Kd_batch_numpy(seqs_np, self.ace2_vector, log10=True) * ln10
        kdace2_np = np.clip(np.squeeze(kdace2_np), -15.0, -5.0)
        kdace2 = torch.as_tensor(kdace2_np, dtype=self.dtype, device=self.device)

        conc = self.raw_concentrations * ln10
        def softplus_stable(x):
            out = torch.empty_like(x)
            m = x > 1
            out[m] = x[m] + torch.log1p(torch.exp(-x[m]))
            out[~m] = torch.log1p(torch.exp(x[~m]))
            return out

        energy = softplus_stable(-kds + conc).sum(dim=-1)
        energy = energy + softplus_stable(kdace2 - self.raw_ace2)

        fe = self._rbm_free_energy(seqs_np)
        energy = (energy + beta * fe) * total_beta
        return energy[0] if single else energy

# ===== Training with BCE on sigmoid(score - WT) =====
def fit_escape_map_sigmoid_diff(
    seqs, y, rbm, kd_vectors, ace2_vector,
    epochs=50, batch_size=512, lr=2e-1, weight_decay=1e-5,
    device=None, dtype=torch.float64
):
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    X = torch.as_tensor(seqs, dtype=torch.long, device=device)
    Y = torch.as_tensor(y.astype(float), dtype=dtype, device=device)

    model = EscapeMapTorch(
        rbm=rbm, kd_vectors=kd_vectors, ace2_vector=ace2_vector,
        total_beta=1.0, device=device, dtype=dtype
    ).to(device)

    # Optimizer: now include raw_concentrations
    params = [model.raw_concentrations, model.raw_beta, model.raw_ace2, model.raw_total_beta]
    opt = torch.optim.Adam(params, lr=lr, weight_decay=weight_decay)

    # Class imbalance handling
    pos = float(Y.sum().item()); neg = float(Y.numel() - pos)
    pos_weight = torch.tensor(neg / max(pos, 1.0), dtype=dtype, device=device)
    bce = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    N = X.size(0); idx_all = torch.arange(N, device=device)
    WT = torch.as_tensor(WT_SEQ, dtype=torch.long, device=device)

    for _ in range(epochs):
        perm = idx_all[torch.randperm(N, device=device)]
        for bi in range(ceil(N / batch_size)):
            bidx = perm[bi * batch_size : (bi + 1) * batch_size]
            xb, yb = X[bidx], Y[bidx]

            opt.zero_grad()
            wt_score = -model(WT)               # scalar
            scores_b = -model(xb)               # [B]
            logits = scores_b - wt_score        # classification margin vs WT
            loss = bce(logits, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(params, max_norm=10.0)
            opt.step()
            print('model.raw_beta:', model.raw_beta.item())
            print('model.raw_ace2:', model.raw_ace2.item())

    # Final evaluation
    with torch.no_grad():
        wt_score = -model(WT)
        scores = -model(X)
        logits_full = scores - wt_score
        prob = torch.sigmoid(logits_full).cpu().numpy()
        auc = roc_auc_score(y, prob)
        rc = model.raw_concentrations.detach().cpu().numpy()
        rb = float(model.raw_beta.detach().cpu())
        ra = float(model.raw_ace2.detach().cpu())
        tb = float(torch.exp(model.raw_total_beta).detach().cpu())  # total_beta in linear scale

    return {
        "model": model,
        "prob": prob,
        "AUC": auc,
        "raw_concentrations": rc,
        "raw_beta": rb,
        "raw_ace2": ra,
        "total_beta": tb,
    }


out = fit_escape_map_sigmoid_diff(
    seqs=seqs, y=Y_ALL, rbm=RBM, kd_vectors=KD_VECTORS, ace2_vector=ACE2_KD_VECTOR,
    epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5,
)
print(out["AUC"], out["raw_ace2"], out["raw_beta"], out["total_beta"], out["raw_concentrations"])
# --- save scores ---
scores_df = pd.DataFrame({
    "seq_index": np.arange(len(seqs)),
    "y": Y_ALL,
    "score": out["prob"]
})
scores_df.to_csv("scores_sigmoid_diff.csv", index=False)

# --- save parameters ---
ab_names = list(KD_VECTORS.keys())
param_row = {
    "AUC": out["AUC"],
    "raw_beta": out["raw_beta"],
    "raw_ace2": out["raw_ace2"],
    "total_beta": out["total_beta"],
}
for name, val in zip(ab_names, out["raw_concentrations"]):
    param_row[f"raw_c_{name}"] = float(val)

params_df = pd.DataFrame([param_row])
params_df.to_csv("params_sigmoid_diff.csv", index=False)




# out = fit_escape_map_sigmoid_diff(
#     seqs=seqs, y=Y_ALL, rbm=RBM, kd_vectors=kd_vector_small, ace2_vector=ACE2_KD_VECTOR,
#     epochs=30, batch_size=2048, lr=2e-1, weight_decay=1e-5,
# )
# print(out["AUC"], out["raw_ace2"], out["raw_beta"], out["total_beta"], out["raw_concentrations"])
# # --- save scores ---
# scores_df = pd.DataFrame({
#     "seq_index": np.arange(len(seqs)),
#     "y": Y_ALL,
#     "score": out["prob"]
# })
# scores_df.to_csv("scores_sigmoid_diff_4ab.csv", index=False)

# # --- save parameters ---
# ab_names = list(kd_vector_small.keys())
# param_row = {
#     "AUC": out["AUC"],
#     "raw_beta": out["raw_beta"],
#     "raw_ace2": out["raw_ace2"],
#     "total_beta": out["total_beta"],
# }
# for name, val in zip(ab_names, out["raw_concentrations"]):
#     param_row[f"raw_c_{name}"] = float(val)

# params_df = pd.DataFrame([param_row])
# params_df.to_csv("params_sigmoid_diff_4ab.csv", index=False)

model.raw_beta: -0.19999999959827966
model.raw_ace2: -9.800001329683786
model.raw_beta: -0.39381662073214574
model.raw_ace2: -9.600973365711658
model.raw_beta: -0.576435298070775
model.raw_ace2: -9.412372952888745
model.raw_beta: -0.7502150356502543
model.raw_ace2: -9.27784720658536
model.raw_beta: -0.9101093962860822
model.raw_ace2: -9.211237075712436
model.raw_beta: -1.0557353721946743
model.raw_ace2: -9.139316612447042
model.raw_beta: -1.187834858498259
model.raw_ace2: -9.113545144343853
model.raw_beta: -1.306368912254779
model.raw_ace2: -9.14178566210178
model.raw_beta: -1.4130802578024997
model.raw_ace2: -9.2052512937526
model.raw_beta: -1.5091549220368636
model.raw_ace2: -9.287907515933846
model.raw_beta: -1.595649662132188
model.raw_ace2: -9.393463866338065
model.raw_beta: -1.6735845395949631
model.raw_ace2: -9.517157427731116
model.raw_beta: -1.7437985366527329
model.raw_ace2: -9.654940722844255
model.raw_beta: -1.8073031551864653
model.raw_ace2: -9.808953038848067
model.raw_be

In [23]:
# import numpy as np
# import pandas as pd
# import torch
# import torch.nn.functional as F
# from sklearn.metrics import roc_auc_score
# from math import ceil

# # pairwise AUC surrogate
# def pairwise_logistic_loss(scores: torch.Tensor, y: torch.Tensor, margin: float = 0.0, max_pairs: int = 200_000):
#     pos = scores[y == 1]
#     neg = scores[y == 0]
#     if pos.numel() == 0 or neg.numel() == 0:
#         return None
#     num_pairs = pos.numel() * neg.numel()
#     if num_pairs > max_pairs:
#         k = max_pairs
#         ip = torch.randint(0, pos.numel(), (k,), device=scores.device)
#         ineg = torch.randint(0, neg.numel(), (k,), device=scores.device)
#         diff = pos[ip] - neg[ineg]
#     else:
#         diff = pos[:, None] - neg[None, :]
#     return F.softplus(-(diff - margin)).mean()

# bce = torch.nn.BCEWithLogitsLoss()

# def optimize_for_check_dates(
#     df, seqs,
#     check_prefix="y_", lr=2e-1, weight_decay=1e-5,
#     epochs=30, batch_size=1024, use_pairwise=True,
#     device=None, dtype=torch.float64,
#     params_csv="fitted_params.csv", scores_csv="scores_by_date.csv"
# ):
#     device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     check_cols = sorted([c for c in df.columns if c.startswith(check_prefix)])
#     ab_names = list(KD_VECTORS.keys())

#     X = torch.as_tensor(seqs, dtype=torch.long, device=device)

#     param_rows = []
#     score_rows = []

#     for col in check_cols:
#         y = Y_ALL
#         if y.sum() == 0 or y.sum() == y.size:
#             print(f"{col}: skipped (single class)")
#             continue

#         Y = torch.as_tensor(y, dtype=dtype, device=device)

#         model = EscapeMapTorch(
#             rbm=RBM, kd_vectors=KD_VECTORS, ace2_vector=ACE2_KD_VECTOR,
#             raw_concentrations=None, raw_ace2=None, raw_beta=None,
#             total_beta=1.0, device=device, dtype=dtype
#         ).to(device)

#         opt = torch.optim.Adam(
#             [model.raw_concentrations, model.raw_beta, model.raw_ace2],
#             lr=lr, weight_decay=weight_decay
#         )

#         N = X.size(0)
#         idx_all = torch.arange(N, device=device)

#         for _ in range(epochs):
#             perm = idx_all[torch.randperm(N, device=device)]
#             for bi in range(ceil(N / batch_size)):
#                 bidx = perm[bi * batch_size : (bi + 1) * batch_size]
#                 xb = X[bidx]
#                 yb = Y[bidx]

#                 opt.zero_grad()
#                 scores_b = -model(xb)
#                 if use_pairwise:
#                     loss = pairwise_logistic_loss(scores_b, yb, margin=0.0)
#                     if loss is None:
#                         loss = bce(scores_b, yb)
#                 else:
#                     loss = bce(scores_b, yb)
#                 loss.backward()
#                 torch.nn.utils.clip_grad_norm_(
#                     [model.raw_concentrations, model.raw_beta, model.raw_ace2], max_norm=10.0
#                 )
#                 opt.step()

#         with torch.no_grad():
#             scores_full = -model(X).cpu().numpy()
#             auc = roc_auc_score(y, scores_full)
#             rc = model.raw_concentrations.detach().cpu().numpy()
#             rb = float(model.raw_beta.detach().cpu())
#             ra = float(model.raw_ace2.detach().cpu())

#         gm_c = float(10.0 ** (np.mean(rc)))
#         gm_beta = float(np.exp(rb))
#         check_date = col[len(check_prefix):]

#         # parameters row
#         row = {
#             "check_date": check_date,
#             "raw_beta": rb,
#             "raw_ace2": ra,
#             "AUC": auc,
#         }
#         # per-antibody raw_c columns
#         for name, val in zip(ab_names, rc):
#             row[f"raw_c_{name}"] = float(val)
#         param_rows.append(row)

#         # scores rows
#         score_rows.extend(
#             {"check_date": check_date, "seq_index": int(i), "score": float(s), "y": int(y[i])}
#             for i, s in enumerate(scores_full)
#         )

#         print(f"{col}: AUC={auc:.4f} raw_ace2={ra:.6g} gm_c={gm_c:.6g} gm_beta={gm_beta:.6g}")
#         break

#     params_df = pd.DataFrame(param_rows)
#     scores_df = pd.DataFrame(score_rows)

#     params_df.to_csv(params_csv, index=False)
#     scores_df.to_csv(scores_csv, index=False)

#     return params_df, scores_df

# # Example:
# params_df, scores_df = optimize_for_check_dates(df_out, seqs, params_csv="params_all.csv", scores_csv="scores_all.csv")
